In [1]:
import antimeridian
import geopandas as gpd
from odc.geo import XY, BoundingBox
from odc.geo.gridspec import GridSpec
from shapely.geometry import shape

In [2]:
# Create a gridspec
pacific_epsg = "EPSG:3832"
# The origin is in the projected CRS. This works for Landsat.
pacific_gridspec = GridSpec(crs=pacific_epsg, tile_shape=(3200,3200), resolution=30, origin=XY(-3000000,-4000000))

# This grid is for Sentinel-2 and has the same footprint
# pacific_gridspec_s2 = GridSpec(crs=pacific_epsg, tile_shape=(9600,9600), resolution=10, origin=XY(-3000000,-4000000))
bounds = BoundingBox(120, -30, 280, 30, crs="EPSG:4326").to_crs(pacific_epsg)

tiles = pacific_gridspec.tiles(bounds)

#geometry, index = zip(*[(a_tile[1].boundingbox.polygon.geom, a_tile[0]) for a_tile in tiles])

# GeoSeries in native projection, should not have rounding issues like the geojson version
# Also sets series index by tile row, column so will work with existing code elsewhere
#gs = gpd.GeoSeries(geometry, index, crs=pacific_epsg)
next(tiles)



((-4, 5),
 GeoBox((3200, 3200), Affine(30.0, 0.0, -3384000.0,
        0.0, -30.0, -3424000.0), CRS('EPSG:3832')))

In [3]:
pacific_gridspec_s2 = GridSpec(crs=pacific_epsg, tile_shape=(9600,9600), resolution=10, origin=XY(-3000000,-4000000))
tiles = pacific_gridspec.tiles(bounds)

geometry, index = zip(*[(a_tile[1].boundingbox.polygon.geom, a_tile[0]) for a_tile in tiles])

# GeoSeries in native projection, should not have rounding issues like the geojson version
# Also sets series index by tile row, column so will work with existing code elsewhere
gs = gpd.GeoSeries(geometry, index, crs=pacific_epsg)
gs

-4    5     POLYGON ((-3384000.000 -3520000.000, -3384000....
-3    5     POLYGON ((-3288000.000 -3520000.000, -3288000....
-2    5     POLYGON ((-3192000.000 -3520000.000, -3192000....
-1    5     POLYGON ((-3096000.000 -3520000.000, -3096000....
 0    5     POLYGON ((-3000000.000 -3520000.000, -3000000....
                                  ...                        
 177  77    POLYGON ((13992000.000 3392000.000, 13992000.0...
 178  77    POLYGON ((14088000.000 3392000.000, 14088000.0...
 179  77    POLYGON ((14184000.000 3392000.000, 14184000.0...
 180  77    POLYGON ((14280000.000 3392000.000, 14280000.0...
 181  77    POLYGON ((14376000.000 3392000.000, 14376000.0...
Length: 13578, dtype: geometry

In [33]:
pacific_admin = (
    gpd.read_file("data/pacific_admin.gpkg")
    .to_crs(pacific_epsg)
    .rename(columns=dict(ISO2="country_code"))
    .drop(columns=["NAME", "osm_id", "layer"])
)

def join_grid(grid, areas_of_interest):
    output = gpd.sjoin(gpd.GeoDataFrame(geometry=gs), areas_of_interest).drop(columns=["index_right"])
    output["bbox"] = output["geometry"].apply(lambda x: ",".join([f"{b:6.0f}" for b in x.bounds]))
    return output

In [27]:


#grid_4326 = grid
#grid_4326.geometry = grid.to_crs(4326).geometry.apply(lambda geom: shape(antimeridian.fix_shape(geom)))

pacific_admin_grid = join_grid(gs, pacific_admin)
pacific_admin_grid.groupby([pacific_admin_grid.index]).first()


,geometry,country_code,bbox
"(9, 45)","POLYGON ((-2136000.000 320000.000, -2136000.00...",PW,"-2136000,320000,-2040000,416000"
"(10, 45)","POLYGON ((-2040000.000 320000.000, -2040000.00...",PW,"-2040000,320000,-1944000,416000"
"(10, 46)","POLYGON ((-2040000.000 416000.000, -2040000.00...",PW,"-2040000,416000,-1944000,512000"
"(10, 47)","POLYGON ((-2040000.000 512000.000, -2040000.00...",PW,"-2040000,512000,-1944000,608000"
"(12, 49)","POLYGON ((-1848000.000 704000.000, -1848000.00...",PW,"-1848000,704000,-1752000,800000"
...,...,...,...
"(118, 14)","POLYGON ((8328000.000 -2656000.000, 8328000.00...",PF,"8328000,-2656000,8424000,-2560000"
"(123, 11)","POLYGON ((8808000.000 -2944000.000, 8808000.00...",PN,"8808000,-2944000,8904000,-2848000"
"(123, 13)","POLYGON ((8808000.000 -2752000.000, 8808000.00...",PN,"8808000,-2752000,8904000,-2656000"
"(125, 12)","POLYGON ((9000000.000 -2848000.000, 9000000.00...",PN,"9000000,-2848000,9096000,-2752000"


In [36]:
pacific_gadm = gpd.read_file('data/gadm_pacific.gpkg').to_crs(3832)
pacific_gadm_grid = join_grid(gs, pacific_gadm)
pacific_gadm_grid.groupby([pacific_gadm_grid.index]).first()
pacific_gadm


,GID_0,COUNTRY,geometry
0,ASM,American Samoa,"MULTIPOLYGON (((4377011.286 -1596109.300, 4377..."
1,COK,Cook Islands,"MULTIPOLYGON (((5596005.799 -2408023.965, 5595..."
2,FJI,Fiji,"MULTIPOLYGON (((3128781.564 -2159676.676, 3128..."
3,PYF,French Polynesia,"MULTIPOLYGON (((7403804.341 -3216412.204, 7403..."
4,GUM,Guam,"MULTIPOLYGON (((-596142.590 1476823.610, -5961..."
5,KIR,Kiribati,"MULTIPOLYGON (((7448332.981 -2006885.947, 7448..."
6,MHL,Marshall Islands,"MULTIPOLYGON (((2465539.148 654152.027, 246554..."
7,FSM,Micronesia,"MULTIPOLYGON (((532231.065 113651.985, 532231...."
8,NRU,Nauru,"MULTIPOLYGON (((1884354.498 -61218.755, 188425..."
9,NCL,New Caledonia,"MULTIPOLYGON (((1878886.656 -2597069.411, 1878..."


In [ ]:
pacific_gadm_grid = join_grid

In [48]:
grid_4326.to_file("grid_pacific.geojson")
grid_3832.to_file("grid.gpkg")